In [5]:
import numpy as np

def local_thresholding(img, window_size):
    half_window = window_size // 2
    top_rows = img[:half_window, :][::-1, :]
    bottom_rows = img[-half_window:, :][::-1, :]
    #Matrix zusammenfügen
    imgM = np.concatenate((top_rows, img, bottom_rows), axis=0)
    
    # Äußere Spalten spiegeln und erweitern
    left_cols = imgM[:, :half_window][:, ::-1]
    right_cols = imgM[:, -half_window:][:, ::-1]
    imgM = np.concatenate((left_cols, imgM, right_cols), axis=1)
    imgT = imgM.copy()
    height, width = imgM.shape[:2]

    for y in range(half_window, height - half_window):
        for x in range(half_window, width - half_window):
            window = imgM[y - half_window : y + half_window + 1, x - half_window : x + half_window + 1]
             
            rvl = window.ravel()
            ran = round(max(rvl) - min(rvl))
            counts, bins = np.histogram(rvl, bins=ran)
            
 
            variance_list = []
 
            for T in range(1, len(counts)):
                sum_back = 0 
                mean_sum_back = 0
                for i in range(0, T):
                    sum_back += counts[i]
                    mean_sum_back += counts[i] * bins[i]
 
                w0 = sum_back / sum(counts)
                mean_back = mean_sum_back / sum_back
 
                sum_obj = 0
                mean_sum_obj = 0
                for j in range(T, len(counts)):
                    sum_obj += counts[j]
                    mean_sum_obj += counts[j] * bins[j]
 
                w1 = sum_obj / sum(counts)
                mean_obj = mean_sum_obj / sum_obj
 
                var_sum_back = 0
                for k in range(0, T):
                    var_sum_back += counts[k] * (bins[k] - mean_back)**2 
 
                var_back = var_sum_back / sum_back
 
                var_sum_obj = 0
                for l in range(T, len(counts)):
                    var_sum_obj += counts[l] * (bins[l] - mean_obj)**2 
 
                var_obj = var_sum_obj / sum_obj
 
                within_class_varianz = w0 * var_back + w1 * var_obj
                variance_list.append(within_class_varianz)
 
            minvar = min(variance_list)
            THRESH = round(bins[variance_list.index(minvar)])
 
            if imgM[y, x] < THRESH:
                imgT[y, x] = 0
            else:
                imgT[y, x] = 255

    return imgT

In [1]:
#local thresholding Vektorisiert

import numpy as np

def local_thresholding_vek(img, window_size):
    half_window = window_size // 2
    top_rows = img[:half_window, :][::-1, :]
    bottom_rows = img[-half_window:, :][::-1, :]
    imgM = np.concatenate((top_rows, img, bottom_rows), axis=0)
    
    left_cols = imgM[:, :half_window][:, ::-1]
    right_cols = imgM[:, -half_window:][:, ::-1]
    imgM = np.concatenate((left_cols, imgM, right_cols), axis=1)
    imgT = imgM.copy()
    height, width = imgM.shape[:2]

    for y in range(half_window, height - half_window):
        for x in range(half_window, width - half_window):
            window = imgM[y - half_window : y + half_window + 1, x - half_window : x + half_window + 1]
             
            rvl = window.ravel()
            ran = round(max(rvl) - min(rvl))
            counts, bins = np.histogram(rvl, bins=ran)
            
            sum_back = np.cumsum(counts)[:-1]
            sum_obj = sum(counts)-sum_back
            mean_back = np.cumsum(bins[:-2]*counts[:-1])/sum_back
            mean_obj = (sum(bins[:-2]*counts[:-1])-np.cumsum(bins[:-2]*counts[:-1]))/sum_obj
            w0_list = sum_back/sum(counts)
            w1_list = 1 - w0_list

            #calculate background and foreground variance
            var_back_list = list()
            var_obj_list = list()
            for i in range (0,len(mean_back)):
                var_back = sum(counts[:i]*(bins[:i]-mean_back[i])**2)/sum_back[i]
                var_back_list.append(var_back)
                var_obj = sum(counts[i:]*(bins[i:-1]-mean_obj[i])**2)/sum_obj[i]
                var_obj_list.append(var_obj)
        
   
        
            wcv_list = (w0_list * var_back_list) + (w1_list * var_obj_list)
            minvar = min(wcv_list)
            for m in range(0,len(wcv_list)):
                if wcv_list[m] == minvar: 
                    THRESH = round(bins[m])
            
 
            if imgM[y, x] < THRESH:
                imgT[y, x] = 0
            else:
                imgT[y, x] = 255

    return imgT


In [3]:
from skimage import io
io.imshow(local_thresholding_vek(io.imread('../BilderDaten/BilderDaten/N2DL-HeLa/img/t13.tif'),window_size = 201))



KeyboardInterrupt: 

In [3]:
#Versucherweiterung Matrix

import numpy as np

def spiegel_matrix(matrix):
    rows, cols = matrix.shape

    # Äußere Zeilen spiegeln und erweitern
    top_rows = matrix[:3, :][::-1, :]
    bottom_rows = matrix[-3:, :][::-1, :]
    #Matrix zusammenfügen
    matrix = np.concatenate((top_rows, matrix, bottom_rows), axis=0)
    
    print(top_rows)
    print(bottom_rows)
    

    # Äußere Spalten spiegeln und erweitern
    left_cols = matrix[:, :3][:, ::-1]
    right_cols = matrix[:, -3:][:, ::-1]
    matrix = np.concatenate((left_cols, matrix, right_cols), axis=1)

    return matrix

# Beispielmatrix
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])

# Matrix spiegeln und erweitern
#neue_matrix = spiegel_matrix(matrix)

print(neue_matrix)


[[1 2 3 1 2 3]
 [4 5 6 4 5 6]
 [7 8 9 7 8 9]]
